# Tarea 3.3 - Modelos de Inteligencia Artificial

In [ ]:
%pip install pandas numpy spacy scikit-learn gensim
!python -m spacy download es_core_news_sm

In [5]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora

Cargamos los datos después de las importaciones

In [6]:
df = pd.read_csv(
    'corpus_mundo_today.csv',
    sep=r'\|\|',
    engine='python',
    encoding='utf-8'
)

df.drop(columns=["tema"], inplace=True)
columnas = df['título'] + ". " + df['texto']
docs_list = columnas.tolist()

Posteriormente normalizamos los datos


In [ ]:
nlp = spacy.load('es_core_news_sm')

def Normalization(docs_list):
    """Normaliza documentos: tokenización, eliminación stopwords, lematización y filtrado POS"""
    corpus_limpio = []
    for documento in nlp.pipe(docs_list, disable=["ner", "parser"]):
        lista_tokens = [
            token.lemma_.lower().strip()
            for token in documento
            if not token.is_punct
            and not token.is_stop
            and token.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADV', 'ADJ']
            and token.text.strip() != ""
        ]
        corpus_limpio.append(" ".join(lista_tokens))
    return corpus_limpio

corpus = Normalization(docs_list)

#### 6. **Crear una función que dada una lista de documentos (*corpus*) tokenizados, elimine del corpus aquellos tokens que aparecen menos de 'N' veces (N=10) en el corpus**

* **input**: lista de listas, en la que cada lista contiene los tokens del documento normalizados.
* **input**: 'N' -> Parámetro que nos indica el número mínimo de apariciones de la palabra en el corpus.
* **output**: lista de listas, en la que cada lista contiene los tokens del documento normalizados.

In [13]:
def drop_less_frecuency_words(corpus, n):
    """Elimina tokens que aparecen menos de n veces en el corpus"""
    # Primero contabilizamos todas las apariciones
    all_tokens = []
    for doc in corpus:
        all_tokens.extend(doc.split())
    
    frecuencias = Counter(all_tokens)

    # Mantenemos solo tokens con frecuencia mayor o igual a n
    tokens_validos = {token for token, freq in frecuencias.items() if freq >= n}
    
    # Por último filtramos corpus
    corpus_filtrado = []
    for doc in corpus:
        tokens_filtrados = [token for token in doc.split() if token in tokens_validos]
        corpus_filtrado.append(" ".join(tokens_filtrados))
    
    return corpus_filtrado

corpus = drop_less_frecuency_words(corpus, 10)
print("Ejemplo documento filtrado (tokens con freq >= 10):")
print(corpus[0][:200] + "...\n")

Ejemplo documento filtrado (tokens con freq >= 10):
gobierno español sumar puigdemont gobierno españa puigdemont semana sumar líder cataluña puigdemont asegurar catalán quedar líder año seguir año puigdemont hacer él semana...



#### 7. **Dado el corpus, normalizado y con tokens que aparecen 10 veces o más en el corpus, se pide crear una bolsa de palabras en ONE-HOT-ENCODE con Gensim**

In [ ]:
corpus_tokenizado = [doc.split() for doc in corpus]

# Creamos el diccionario de términos
diccionario = corpora.Dictionary(corpus_tokenizado)

print(f"Número de tokens únicos: {len(diccionario)}")
print("\nPrimeros 10 tokens del diccionario:")
for i, token in list(diccionario.items())[:10]:
    print(f"{i}: {token}")

# Convertimos a Bag of Words
corpus_bow = [diccionario.doc2bow(doc) for doc in corpus_tokenizado]

print("\nBoW del primer documento (One-Hot):")
print(corpus_bow[0][:10])

Número de tokens únicos: 134

Primeros 10 tokens del diccionario:
0: asegurar
1: año
2: cataluña
3: catalán
4: españa
5: español
6: gobierno
7: hacer
8: líder
9: puigdemont

BoW del primer documento (One-Hot):
[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 2), (9, 4)]


#### 8. **Dado el corpus, normalizado y con tokens que aparecen 10 veces o más en el corpus, se pide crear una bolsa de palabras aplicando el TF-IDF con Scikit**

In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print("\n\nTF-IDF Matrix:")
print(f"Shape: {tfidf_matrix.shape}")
print(f"Vocabulario: {len(tfidf_vectorizer.vocabulary_)} términos")

print("\nPrimeros 10 términos del vocabulario:")
vocab = list(tfidf_vectorizer.vocabulary_.items())[:10]
for term, idx in vocab:
    print(f"{term}: índice {idx}")

print("\nTF-IDF del primer documento (primeros 10 valores):")
doc_tfidf = tfidf_matrix[0].toarray()[0]
indices_ordenados = np.argsort(doc_tfidf)[::-1][:10]
feature_names = tfidf_vectorizer.get_feature_names_out()
for idx in indices_ordenados:
    if doc_tfidf[idx] > 0:
        print(f"{feature_names[idx]}: {doc_tfidf[idx]:.4f}")



TF-IDF Matrix:
Shape: (52, 134)
Vocabulario: 134 términos

Primeros 10 términos del vocabulario:
gobierno: índice 54
español: índice 44
sumar: índice 121
puigdemont: índice 106
españa: índice 43
semana: índice 118
líder: índice 75
cataluña: índice 17
asegurar: índice 4
catalán: índice 18

TF-IDF del primer documento (primeros 10 valores):
puigdemont: 0.7460
sumar: 0.3078
líder: 0.2758
semana: 0.2587
gobierno: 0.2137
año: 0.2036
cataluña: 0.1539
hacer: 0.1335
quedar: 0.1255
españa: 0.1124
